In [ ]:
from pathlib import Path
import os
import json
import tqdm
import glob
import SimpleITK as sitk

In [ ]:
# This is a helper script to compute the staple masks used as ground truth for the
# testing sets.

In [ ]:
# Set the GC archive to upload to

# prelimary testing phase
DATASET_DIR = "../dataset/preliminary"

# final testing phase
DATASET_DIR = "../dataset/testing"


In [ ]:
# Provide the expected cases to upload
case_ids = sorted(os.listdir(DATASET_DIR))
case_ids = [case_id for case_id in case_ids if os.path.isdir(f"{DATASET_DIR}/{case_id}")]

In [ ]:
for case_id in tqdm.tqdm(case_ids):
    LABEL_FILES_GLOB = f"{DATASET_DIR}/{case_id}/targets/{case_id}_labels*.mha"
    LABELS = [sitk.ReadImage(f) for f in glob.glob(LABEL_FILES_GLOB)]
    staple_label = sitk.STAPLEImageFilter().Execute(LABELS)
    staple_label = staple_label > 0.5 
    #staple_label = sitk.BinaryThresholdImageFilter().Execute(staple_label)
    staple_label = sitk.Cast(staple_label, sitk.sitkUInt8)
    staple_label.SetSpacing(LABELS[0].GetSpacing())
    staple_label.SetOrigin(LABELS[0].GetOrigin())
    staple_label.SetDirection(LABELS[0].GetDirection())
    sitk.WriteImage(staple_label, f"{DATASET_DIR}/{case_id}/targets/{case_id}_staple_labels.mha", useCompression=True)

In [ ]:
# prepare ground truth archive .tar.gz using python
# only contains
# {case_id}/targets/{case_id}_labels.mha

import tarfile

filename = os.path.join(os.path.dirname(DATASET_DIR), os.path.basename(DATASET_DIR) + "_gt.tar.gz")

with tarfile.open(filename, "w:gz") as tar:
    for case_id in case_ids:
        target = f"{DATASET_DIR}/{case_id}/targets/{case_id}_staple_labels.mha"
        if not os.path.exists(target):
            raise RuntimeError(f"Could not find {target}")
        tar.add(target, arcname=f"{case_id}/targets/{case_id}_labels.mha")


In [ ]:
# check if the tar.gz is correct and matches the full dataset
with tarfile.open(filename, "r:gz") as tar:
    for member in tar.getmembers():
        print(member.name, member.size)
        case_id = member.name.split("/")[0]
        !ls -l $DATASET_DIR/{case_id}/targets/{case_id}_labels.mha